In [3]:
import os 
import openai
import pandas as pd

OPENAI_API_KEY = ""
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = OPENAI_API_KEY
model1 = 'gpt-3.5-turbo-16k' 
model2 = 'gpt-4-0613'

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.llms.loading import load_llm
from langchain.callbacks import get_openai_callback
from langchain.agents import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
S
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory

from langchain.vectorstores import Chroma
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


llm = OpenAI(temperature=0, model=model2)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.6) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [5]:
# MAIN CODE FOR OBJECTIVE 1##
# USE FOR MULTIPLE PERIODS, MULTIPLE PORTFOLIOS
# Multiple prompts
# with CSV output with regular and strat samples
#3 placeholders


from datetime import datetime
import pandas as pd




#file names setup
#inputs
#strat_file="./Data/Attribution/strat_attrib_synt.csv"
regular_file = './Data/Attribution/attribution_by_gics_Obj1.csv'

#input in use
file_insue = regular_file



import csv
from io import StringIO


#prompt functions that capture more fields from the csv file



def construct_prompt_zeroshot_2(sector):
    
    prompt = f"""
    Analyze performance attribution for the {sector} sector using the Brinson model. 
    Provide:
    1. Two bullet points: one for allocation and one for selection, with reasoning and numerical evidence.
    2. Convert the bullet points in a CSV format with columns: Sector, Effect Type, Value, Sector Weight, Sector Performance.
    3. CSV column "Sector Weight" choices: Underweight or Overweight. On selection, write None.
    4. CSV column "Sector Performance" choices: Outperformance or Underperformance.
    - Output should contain only bullet points and the CSV format, separate by 'CSV Format:'.
    - Write maximum 3 sentences per bullet point.
    
    """
    return prompt

def construct_prompt_fewshot_1_new(sector):
    
    prompt = f"""
    Analyze performance attribution for the {sector} sector using the Brinson model.
    Provide:
    1. Two bullet points: one for allocation and one for selection, with reasoning and numerical evidence.
    2. Convert the bullet points in a CSV format with columns: Sector, Effect Type, Value, Sector Weight, Sector Performance.
    3. CSV column "Sector Weight" choices extracted from the bullet points: Underweight, Overweight or Neutral. On selection, write None.
    4. CSV column "Sector Performance" choices extracted from the bullet points: Outperformance, Underperformance or Neutral.
    - Output should contain only bullet points and the CSV format, separate by 'CSV Format:'
    - Write in a format that does not deviate from the following two sentences - choose the words in brackets observing and reasoning on the data provided - pay close 
    attention to the comparison between weights and returns and how that affects the right choice of words:
    - The 'X' sector had a '(positive/negative/zero)' allocation effect of 'y'. This was due to the fund being '(overweight/underweight/neutral)' in this sector compared to 
    the benchmark and due to the fact that the benchmark return of 'a' compared to the the benchmark total return of 'b' '(outperformed/underperformed)'.
    - The 'X' sector also had a '(positive/negative/zero)' selection effect of 'x'. The fund's investments in this sector '(outperformed/underperformed/neutral)' compared to 
    the sector benchmark (Portfolio Return: 'c' vs Benchmark Return: 'd').
    Attention: 'Benchmark Total Return' is calculated as the weighted average between columns 'Benchmark weight' and 'Benchmark return'. Calculate it.
    Attention: Please only output should be the two bullet points starting with "-" and the CSV format, separated by 'CSV Format:'
    """
    return prompt

def construct_prompt_fewshot_2_new(sector):
    
    prompt = f"""
    Analyze performance attribution for the {sector} sector using the Brinson model.
    Provide:
    1. Two bullet points: one for allocation and one for selection, with reasoning and numerical evidence.
    2. Convert the bullet points in a CSV format with columns: Sector, Effect Type, Value, Sector Weight, Sector Performance.
    3. CSV column "Sector Weight" choices extracted from the bullet points: Underweight, Overweight or Neutral. On selection, write None.
    4. CSV column "Sector Performance" choices extracted from the bullet points: Outperformance, Underperformance or Neutral.
    - Output should contain only bullet points and the CSV format, separate by 'CSV Format:'
    - Write in a format that does not deviate from the following two sentences - choose the words in brackets observing and reasoning on the data provided - pay close attention to the comparison between weights and returns and how that affects the right choice of words:
    - The 'X' sector had a '(positive/negative/zero)' allocation effect of 'y'. This was due to the fund being '(overweight/underweight/neutral)' in this sector compared to the benchmark and due to 
    the fact that the benchmark return of 'a' compared to the the benchmark total return of 'b' '(outperformed/underperformed)'.
    - The 'X' sector also had a '(positive/negative/zero)' selection effect of 'x'. The fund's investments in this sector '(outperformed/underperformed/neutral)' compared to the sector benchmark 
    (Portfolio Return: 'c' vs Benchmark Return: 'd').
    Attention: 'Benchmark Total Return' is calculated as the weighted average between columns 'Benchmark weight' and 'Benchmark return'. Calculate it.
   
   Attention: These additional explanations helps with your reasoning, especially in the allocation effect:
    Positive Allocation occurs in two instances: 1) when the portfolio is 'overweigthed' versus the benchmark and the benchmark return 'outperform's the benchmark total return
    or 2) when the portfolio is 'underweighted' versus the benchmark and the benchmark return 'underperforms' the benchmark total return.
    Negative allocation occurs in two instances: 1) when the portfolio is 'overweigthed' versus the benchmark and the benchmark return 'underperforms' the benchmark total 
    return or 2) when the portfolio is 'underweighted' versus the benchmark and the benchmark return 'outperforms' the benchmark total return.
    Attention: Please only output should be the two bullet points starting with "-" and the CSV format, separated by 'CSV Format:'
    """
    return prompt

def construct_prompt_fewshot_3_new(sector):
    
    prompt = f"""
    Analyze performance attribution for the {sector} sector using the Brinson model.
    Provide:
    1. Two bullet points: one for allocation and one for selection, with reasoning and numerical evidence.
    2. Convert the bullet points in a CSV format with columns: Sector, Effect Type, Value, Sector Weight, Sector Performance.
    3. CSV column "Sector Weight" choices extracted from the bullet points: Underweight, Overweight or Neutral. On selection, write None.
    4. CSV column "Sector Performance" choices extracted from the bullet points: Outperformance, Underperformance or Neutral.
    - Output should contain only bullet points and the CSV format, separate by 'CSV Format:'
    - Write in a format that does not deviate from the following two sentences - choose the words in brackets observing and reasoning on the data provided - pay close attention to the comparison between weights and returns and how that affects the right choice of words:
    - The 'X' sector had a '(positive/negative/zero)' allocation effect of 'y'. This was due to the fund being '(overweight/underweight/neutral)' in this sector compared to the benchmark and due to 
    the fact that the benchmark return of 'a' compared to the the benchmark total return of 'b' '(outperformed/underperformed)'.
    - The 'X' sector also had a '(positive/negative/zero)' selection effect of 'x'. The fund's investments in this sector '(outperformed/underperformed/neutral)' compared to the sector benchmark 
    (Portfolio Return: 'c' vs Benchmark Return: 'd').
    Attention: 'Benchmark Total Return' is calculated as the weighted average between columns 'Benchmark weight' and 'Benchmark return'. Calculate it.
    Attention: These examples helps with your reasoning, especially in the allocation effect - pay close attention to the comparison between weights and returns and how that affects the right choice of words:
    
    - The 'Consumer Discret.' sector had a 'positive' allocation effect of '0.03'. This was due to the fund being 'overweight' in this sector compared to the benchmark and due to the fact that the benchmark return of '0.05' compared
    to the the benchmark total return of '0.04' 'outperformed'.
    - The 'Consumer Discret.' sector had a 'positive' allocation effect of '0.03'. This was due to the fund being 'underweight' in this sector compared to the benchmark and due to the fact that the benchmark return of '0.05' compared
    to the the benchmark total return of '0.06' 'underperformed'.
    - The 'Consumer Discret.' sector had a 'negative' allocation effect of '-0.03'. This was due to the fund being 'overweight' in this sector compared to the benchmark and due to the fact that the benchmark return of '0.05' compared
    to the the benchmark total return of '0.06' 'underperformed'.
    - The 'Consumer Discret.' sector had a 'negative' allocation effect of '-0.03'. This was due to the fund being 'underweight' in this sector compared to the benchmark and due to the fact that the benchmark return of '0.05' compared
    to the the benchmark total return of '0.04' 'outperformed'.    

    Attention: Please only output should be the two bullet points starting with "-" and the CSV format, separated by 'CSV Format:'
    """
    return prompt

# Function to save a CSV format DataFrame to a file
def save_csv_temp(fund, period, data):
    output_file_temp = f"temp_{fund}_{period.replace('/', '_')}.csv"
    with open(output_file_temp, 'w') as f:
        f.write(data)
    #print(f"Results saved to {output_file}")
    return output_file_temp

#df_temp = pd.read_csv(readfile_insue)

df1 = pd.read_csv(file_insue)



# Step 2: Group by 'Fund' and 'Period'
for (fund, period), group in df1.groupby(['Fund', 'Period']):
    # Step 3: Convert the group to a CSV format string 
    csv_data = group.to_csv(index=False)
    temp_file = save_csv_temp(fund, period, csv_data)
    input_df = pd.read_csv(temp_file)
    #agent setup
    memory_chat = ConversationBufferMemory()

    agent = create_csv_agent(
        ChatOpenAI(temperature=0, model=model2),
        temp_file,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        memory=memory_chat
    )
    
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
    #outputs - few shot
    output_csv = f"./Data/Attribution/Eval_Obj_1/output_csv_few_1_new_{fund}_{period.replace('/', '_')}_{timestamp}.csv"
    output_bullet = f"./Data/Attribution/Eval_Obj_1/output_bullet_few_1_new_{fund}_{period.replace('/', '_')}_{timestamp}.csv"
   
    
    # Create or overwrite the output CSV file
    with open(output_csv, 'w') as f:
        f.write("")

    # Initialize an empty list for bullet points
    bullet_points_list = []

    # Ensure the input CSV has data
    if not input_df.empty:
        # Loop through each row in the first column
        bench_total = (input_df["Benchmark Weight"] * input_df["Benchmark Return"]).sum() / (input_df["Benchmark Weight"]).sum()

        for index, row in input_df.iterrows():

            sector = row['GICS Sector']  # Adjust column name if different
            #if sector == 'Total':
            #    continue
            allocation = row['Allocation Effect']  # Adjust column name if different
            selection = row['Selection Effect']  # Adjust column name if different
            pf_wgt = row['Portfolio Weight']
            pf_ret = row['Portfolio Return']
            bmk_wgt = row['Benchmark Weight']
            bmk_ret =row['Benchmark Return']
            fund = row['Fund']
            period= row['Period']

            # Query the GPT-4 API
            #prompt_txt = construct_prompt_fewshot_1(sector, allocation, selection, fund, period)
            prompt_txt = construct_prompt_fewshot_1_new(sector)
            #prompt_txt = construct_prompt_zeroshot_1(sector, allocation, selection, fund, period)
            #prompt_txt =construct_prompt_zeroshot_2(sector)
            response_text = agent.run(prompt_txt)
            #response_text = print(response_text)
            #memory_chat.clear()
            
                # Check if "CSV Format:" is in the response_text
            if "CSV Format:" not in response_text:
            # Optionally, handle the case where the format is not found
                print(f"CSV Format not found in response for index {index}. Skipping.")
                continue  # Skip to the next iteration of the loop
           
            # Separate bullet points and CSV data
            bullet_points, csv_data = response_text.split("CSV Format:", 1)
            bullet_points_list.append(bullet_points.strip())

            # Convert CSV data to DataFrame
            df = pd.read_csv(StringIO(csv_data.strip()))

            #true allocation and selection effects
            #col_index = df.shape[1]  # This gives the current number of columns
            #df.insert(col_index, 'Check', ['overweight', 'underweight']) "True Value", "True Sector Wgt","True Performance"
            df["True Value"] = [allocation, selection]

            #df.iloc[:, 6] = [selection,selection]
            # allocation cases df.iloc[:, 4] = check_values
            if allocation > 0:
                if pf_wgt > bmk_wgt:
                    df["True Sector Weight"] = ['Overweight','']
                    if selection > 0:
                        df["True Performance"] = ['Outperformance','Outperformance']
                    elif selection < 0:
                        df["True Performance"] = ['Outperformance','Underperformance']
                    else:
                        df["True Performance"] = ['Outperformance','Neutral']
                elif pf_wgt < bmk_wgt:
                    df["True Sector Weight"] = ['Underweight','']
                    if selection > 0:
                        df["True Performance"] = ['Underperformance','Outperformance']
                    elif selection < 0:
                        df["True Performance"] = ['Underperformance','Underperformance']
                    else:
                        df["True Performance"] = ['Underperformance','Neutral']
            elif allocation < 0:
                if pf_wgt > bmk_wgt:
                    df["True Sector Weight"] = ['Overweight','']
                    if selection > 0:
                        df["True Performance"] = ['Underperformance','Outperformance']
                    elif selection < 0:
                        df["True Performance"] = ['Underperformance','Underperformance']
                    else:
                        df["True Performance"] = ['Underperformance','Neutral']
                elif pf_wgt < bmk_wgt:
                    df["True Sector Weight"] = ['Underweight','']
                    if selection > 0:
                        df["True Performance"] = ['Outperformance','Outperformance']
                    elif selection < 0:
                        df["True Performance"] = ['Outperformance','Underperformance']
                    else:
                        df["True Performance"] = ['Outperformance','Neutral']
            elif allocation == 0:
                if pf_wgt == bmk_wgt:
                    df["True Sector Weight"] = ['Neutral','']
                    if bmk_ret > bench_total:
                        if selection > 0:
                            df["True Performance"] = ['Outperformance','Outperformance']
                        elif selection < 0:
                            df["True Performance"] = ['Outperformance','Underperformance']
                        else:
                            df["True Performance"] = ['Outperformance','Neutral']
                    elif bmk_ret < bench_total:
                        if selection > 0:
                            df["True Performance"] = ['Underperformance','Outperformance']
                        elif selection < 0:
                            df["True Performance"] = ['Underperformance','Underperformance']
                        else:
                            df["True Performance"] = ['Underperformance','Neutral']
                    else:
                        if selection > 0:
                            df["True Performance"] = ['Neutral','Outperformance']
                        elif selection < 0:
                            df["True Performance"] = ['Neutral','Underperformance']
                        else:
                            df["True Performance"] = ['Neutral','Neutral']
                elif pf_wgt < bmk_wgt:
                    df["True Sector Weight"] = ['Underweight','']
                    if selection > 0:
                        df["True Performance"] = ['Neutral','Outperformance']
                    elif selection < 0:
                        df["True Performance"] = ['Neutral','Underperformance']
                    else:
                        df["True Performance"] = ['Neutral','Neutral']
                elif pf_wgt > bmk_wgt:
                    df["True Sector Weight"] = ['Overweight','']
                    if selection > 0:
                        df["True Performance"] = ['Neutral','Outperformance']
                    elif selection < 0:
                        df["True Performance"] = ['Neutral','Underperformance']
                    else:
                        df["True Performance"] = ['Neutral','Neutral']            
           
            df["Fund"] = [fund,fund]
            df["Period"] = [period,period]

            # Append data to the output CSV file
            if os.path.exists(output_csv):
                df.to_csv(output_csv, mode='a', header=False, index=False)
            else:
                df.to_csv(output_csv, mode='a', header=True, index=False)

        # Reload the saved dataset
        df_reloaded = pd.read_csv(output_csv, header=None)

        # Define the new headers
        headers = ["Sector", "Effect Type", "Value", "Sector Weight", "Sector Performance","True Value", "True Sector Weight","True Performance","Fund","Period"]
        # Add the headers
        df_reloaded.columns = headers
        # Save the dataset with headers
        file_path_with_headers = output_csv
        df_reloaded.to_csv(file_path_with_headers, index=False)

    else:
        print("The input CSV file is empty.")    

    # Convert the bullet points list to a DataFrame and save
    bullet_points_df = pd.DataFrame(bullet_points_list, columns=["Bullet Points"]) 


    # Convert the bullet points list to a DataFrame
    # Split the "Bullet Points" column into two separate columns based on the newline character
    def split_on_second_sentence(s):
        # Find all occurrences of '- ' in the string
        indices = [i for i, x in enumerate(s) if x == '-' and (i + 1 < len(s) and s[i + 1] == ' ')]

        # If there's more than one occurrence, split at the second occurrence
        if len(indices) > 1:
            return s[:indices[1]].strip(), s[indices[1]:].strip()
        else:
            return s, None

    # Apply the split function to the 'Bullet Points' column
    bullet_points_df['Allocation'], bullet_points_df['Selection'] = zip(*bullet_points_df['Bullet Points'].apply(split_on_second_sentence))

    # Drop the original 'Bullet Points' column
    df_split_corrected = bullet_points_df[['Allocation', 'Selection']]
    # Explicitly create a copy of the DataFrame
    df_split_corrected = df_split_corrected.copy()

    # Now, use .loc[] to make modifications
    df_split_corrected.loc[:, 'Allocation'] = df_split_corrected['Allocation'].str.replace('- ', '', 1)
    df_split_corrected.loc[:, 'Selection'] = df_split_corrected['Selection'].str.replace('- ', '', 1)
    df_split_corrected['Fund'] = fund
    df_split_corrected['Period'] = period

    df_split_corrected.head()
    #df_split_corrected.to_csv('./Data/Attribution/bullet_points_final.csv', index=False)
    df_split_corrected.to_csv(output_bullet, index=False)
    memory_chat.clear()



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "benchmark_total_return = (df['Benchmark Weight'] * df['Benchmark Return']).sum(); benchmark_total_return"}`


-0.05553991999999999- The Communication sector had a negative allocation effect of -0.000288822. This was due to the fund being overweight in this sector compared to the benchmark and due to the fact that the benchmark return of -0.1336 compared to the the benchmark total return of -0.05554 underperformed.
- The Communication sector also had a positive selection effect of 0.00069599. The fund's investments in this sector outperformed compared to the sector benchmark (Portfolio Return: -0.1257 vs Benchmark Return: -0.1336).

CSV Format:
Sector,Effect Type,Value,Sector Weight,Sector Performance
Communication,Allocation,-0.000288822,Overweight,Underperformance
Communication,Selection,0.00069599,None,Outperformance

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `python_repl_

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


- The Cash sector had a negative allocation effect of -0.001185. This was due to the fund being overweight in this sector compared to the benchmark and due to the fact that the benchmark return of 0.0 compared to the the benchmark total return of 0.0273 underperformed.
- The Cash sector also had a positive selection effect of 0.000163. The fund's investments in this sector outperformed compared to the sector benchmark (Portfolio Return: 0.00375 vs Benchmark Return: 0.0).

CSV Format:
Sector,Effect Type,Value,Sector Weight,Sector Performance
Cash,Allocation,-0.001185,Overweight,Underperformance
Cash,Selection,0.000163,None,Outperformance

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "benchmark_total_return = (df['Benchmark Weight'] * df['Benchmark Return']).sum(); benchmark_total_return"}`


0.14885619- The Communication sector had a negative allocation effect of -0.000211838. This was due to the fund being underweight in this se